In [79]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import monotonically_increasing_id, format_number
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import LinearRegression

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import FMRegressor
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.ml import Pipeline

In [80]:
# Создание SparkSession
spark = SparkSession.builder.appName("CrabAgePrediction").getOrCreate()
# Загружаем датасет
data = spark.read.csv("CrabAgePrediction.csv", header = True, inferSchema = True)

data.printSchema()
data.show()

root
 |-- Sex: string (nullable = true)
 |-- Length: double (nullable = true)
 |-- Diameter: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Shucked Weight: double (nullable = true)
 |-- Viscera Weight: double (nullable = true)
 |-- Shell Weight: double (nullable = true)
 |-- Age: integer (nullable = true)

+---+------+--------+------+-----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|     Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+-----------+--------------+--------------+------------+---+
|  F|1.4375|   1.175|0.4125| 24.6357155|    12.3320325|     5.5848515|    6.747181|  9|
|  M|0.8875|    0.65|0.2125| 5.40057975|     2.2963095|    1.37495075|   1.5592225|  6|
|  I|1.0375|   0.775|  0.25| 7.95203475|      3.231843|    1.60174675|  2.76407625|  6|
|  F| 1.175|  0.8875|  0.25|13.48018725|    4.74854125|    2.28213475|   5.2446575| 10|
|  I|0.8875|  0.6

In [81]:
# Преобразуем Пол краба в числа
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
# Преобразовываем данные. Начинаем с выбора признаков
features = ["SexIndex", "Length", "Diameter" , "Height", "Weight", "Shucked Weight", "Viscera Weight", "Shell Weight"]
# Создаем вектор этих признаков
assembler = VectorAssembler(inputCols=features, outputCol="features")
# Разделяем данные на обучение и тест
(train, test) = data.randomSplit([0.7, 0.3])
# Создаем кортеж среднеквадратичных ошибок
tupleRMSE = ()
Pred_arr = []

train.show(3)
test.show(3)

print(train.count())
print(test.count())

+---+------+--------+------+---------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|   Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+---------+--------------+--------------+------------+---+
|  F|0.6875|  0.4875| 0.175|  2.26796|     0.8788345|    0.60951425|   0.7087375|  5|
|  F| 0.725|   0.525|0.1875|7.7961125|     3.2034935|    1.91359125|   0.9922325|  6|
|  F| 0.725|  0.5625|0.1875|  3.96893|    1.45999925|    0.66621325|     1.13398|  5|
+---+------+--------+------+---------+--------------+--------------+------------+---+
only showing top 3 rows

+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F| 0.825|    0.65|   0.2|    5.6699|    1.77184375|      1.417475|    1.984465|  9|
|  F|0.8625|    0.65| 0.2

In [82]:
# Создаем модель
rf = RandomForestRegressor(featuresCol = "features", labelCol= "Age")
# Создаем конвейер
pipelineRF = Pipeline(stages = [indexer, assembler, rf])
# Обучение модели
modelRF = pipelineRF.fit(train)
# Предсказание возраста на тестовой выборке
predictionsRF = modelRF.transform(test).withColumnRenamed("prediction", "predictionsRF")
Pred_arr.append(predictionsRF)

In [83]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsRF", metricName = "rmse")
rmseRF = evaluator.evaluate(predictionsRF)
tupleRMSE += (rmseRF,)
print("Root Mean Squared Error (RMSE): %s" % rmseRF)
# Вывод результата и выключение SparkSession
predictionsRF.select("Age", "predictionsRF").show()

Root Mean Squared Error (RMSE): 2.379645261061958
+---+-----------------+
|Age|    predictionsRF|
+---+-----------------+
|  9|7.451043906487937|
| 10| 7.64939721984125|
|  8|7.774146823544131|
|  9|8.630172027198922|
|  7|   8.186580571906|
|  7|7.858538883841175|
| 10|9.334912359228086|
| 13|8.161428524332003|
| 10|8.168024586146334|
| 16|9.477683834731325|
|  8|9.104203072974553|
| 10|9.378502954914845|
|  8|9.389978080276148|
|  9|9.456581201479818|
| 10|9.518460663548158|
| 12|9.903625363084291|
|  8|9.409185013596904|
|  7|9.355851951310353|
|  7|9.239940508477739|
|  9|9.518460663548158|
+---+-----------------+
only showing top 20 rows



In [84]:
# Создание модели градиентного бустинга
gbt = GBTRegressor(featuresCol="features", labelCol= "Age")
# Создание конвейера
pipelineGBT = Pipeline(stages = [indexer, assembler, gbt])
# Обучение модели
modelGBT = pipelineGBT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGBT = modelGBT.transform(test).withColumnRenamed("prediction", "predictionsGBT")
Pred_arr.append(predictionsGBT)

In [85]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGBT", metricName = "rmse")
rmseGBT = evaluator.evaluate(predictionsGBT)
tupleRMSE += (rmseGBT,)
print("Root Mean Squared Error (RMSE): %s" % rmseGBT)
# Вывод результата и выключение SparkSession
predictionsGBT.select("Age", "predictionsGBT").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.3482925159542054
+---+------------------+
|Age|    predictionsGBT|
+---+------------------+
|  9|  8.44563430533836|
| 10|  8.44563430533836|
|  8|  8.44563430533836|
|  9| 8.543337312434257|
|  7| 8.543337312434257|
|  7| 8.436208073146052|
| 10|  8.64874895739631|
| 13| 8.543337312434257|
| 10| 9.817570685592361|
| 16| 9.851299662316755|
|  8| 9.782124114541814|
| 10| 9.695784699344193|
|  8| 9.782124114541814|
|  9| 9.695784699344193|
| 10| 9.566157006798928|
| 12|10.876863756322242|
|  8| 9.695784699344193|
|  7| 9.695784699344193|
|  7| 9.475723373045145|
|  9| 9.431807599664202|
+---+------------------+
only showing top 20 rows



In [86]:
# Создание модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="Age", maxIter=10, regParam=0.3, elasticNetParam=0.8) 
# Создание конвейера
pipelineLR = Pipeline(stages = [indexer, assembler, lr])
# Fit the model
modelLR = pipelineLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsLR = modelLR.transform(test).withColumnRenamed("prediction", "predictionsLR")
Pred_arr.append(predictionsLR)

In [87]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsLR", metricName = "rmse")
rmseLR = evaluator.evaluate(predictionsLR)
tupleRMSE += (rmseLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseLR)
# Вывод результата и выключение SparkSession
predictionsLR.select("Age", "predictionsLR").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.5874596058672874
+---+------------------+
|Age|     predictionsLR|
+---+------------------+
|  9|  7.53726977356601|
| 10|7.6908128965979214|
|  8|  7.58758583925077|
|  9| 7.954387398416159|
|  7|7.9328861176827825|
|  7| 7.730888397187284|
| 10| 8.262321619306553|
| 13|7.9900794529209955|
| 10| 7.891433780648311|
| 16| 8.331292845724576|
|  8| 8.184233022595023|
| 10| 8.375661163695309|
|  8| 8.235174056900116|
|  9| 8.386204373681018|
| 10| 8.763117813752434|
| 12| 8.803519937521106|
|  8| 8.608732800328166|
|  7| 8.486817522268453|
|  7| 8.533579740048749|
|  9| 8.822097850690803|
+---+------------------+
only showing top 20 rows



In [88]:
# Создание модели решающих деервьев
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineDT = Pipeline(stages = [indexer, assembler, dt])
# Fit the model
modelDT = pipelineDT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsDT = modelDT.transform(test).withColumnRenamed("prediction", "predictionsDT")
Pred_arr.append(predictionsDT)

In [89]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsDT", metricName = "rmse")
rmseDT = evaluator.evaluate(predictionsDT)
tupleRMSE += (rmseDT,)
print("Root Mean Squared Error (RMSE): %s" % rmseDT)
# Вывод результата и выключение SparkSession
predictionsDT.select("Age", "predictionsDT").show()

Root Mean Squared Error (RMSE): 2.4254381741791713
+---+-----------------+
|Age|    predictionsDT|
+---+-----------------+
|  9|8.358024691358025|
| 10|8.358024691358025|
|  8|8.358024691358025|
|  9|8.358024691358025|
|  7|8.358024691358025|
|  7|8.358024691358025|
| 10|8.358024691358025|
| 13|8.358024691358025|
| 10| 9.63225806451613|
| 16| 9.63225806451613|
|  8| 9.63225806451613|
| 10| 9.63225806451613|
|  8| 9.63225806451613|
|  9| 9.63225806451613|
| 10| 9.63225806451613|
| 12| 9.63225806451613|
|  8| 9.63225806451613|
|  7| 9.63225806451613|
|  7| 9.63225806451613|
|  9| 9.63225806451613|
+---+-----------------+
only showing top 20 rows



In [90]:
# Создание модели изотонической регрессии
ir = IsotonicRegression(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineIR = Pipeline(stages = [indexer, assembler, ir])
# Fit the model
modelIR = pipelineIR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsIR = modelIR.transform(test).withColumnRenamed("prediction", "predictionsIR")
Pred_arr.append(predictionsIR)

In [91]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsIR", metricName = "rmse")
rmseIR = evaluator.evaluate(predictionsIR)
tupleRMSE += (rmseIR,)
print("Root Mean Squared Error (RMSE): %s" % rmseIR)
# Вывод результата и выключение SparkSession
predictionsIR.select("Age", "predictionsIR").show()

Root Mean Squared Error (RMSE): 3.996671378313417
+---+------------------+
|Age|     predictionsIR|
+---+------------------+
|  9|10.120329936923824|
| 10|10.120329936923824|
|  8|10.120329936923824|
|  9|10.120329936923824|
|  7|10.120329936923824|
|  7|10.120329936923824|
| 10|10.120329936923824|
| 13|10.120329936923824|
| 10|10.120329936923824|
| 16|10.120329936923824|
|  8|10.120329936923824|
| 10|10.120329936923824|
|  8|10.120329936923824|
|  9|10.120329936923824|
| 10|10.120329936923824|
| 12|10.120329936923824|
|  8|10.120329936923824|
|  7|10.120329936923824|
|  7|10.120329936923824|
|  9|10.120329936923824|
+---+------------------+
only showing top 20 rows



In [92]:
# Создание модели FMRegressor
fm = FMRegressor(featuresCol="features", labelCol="Age", predictionCol="prediction",stepSize=0.001)
# Создание конвейера
pipelineFM = Pipeline(stages = [indexer, assembler, fm])
# Fit the model
modelFM = pipelineFM.fit(train)
# Предсказание возраста на тестовой выборке
predictionsFM = modelFM.transform(test).withColumnRenamed("prediction", "predictionsFM")
Pred_arr.append(predictionsFM)

In [93]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsFM", metricName = "rmse")
rmseFM = evaluator.evaluate(predictionsFM)
tupleRMSE += (rmseFM,)
print("Root Mean Squared Error (RMSE): %s" % rmseFM)
# Вывод результата и выключение SparkSession
predictionsFM.select("Age", "predictionsFM").show()

Root Mean Squared Error (RMSE): 5.474825172045019
+---+------------------+
|Age|     predictionsFM|
+---+------------------+
|  9|1.2982190482582117|
| 10|1.3697069552937873|
|  8|1.3698037940314403|
|  9|1.7496118708984658|
|  7|1.7499938314374526|
|  7|1.8828110530480398|
| 10| 2.062861105183761|
| 13|1.7578563474240785|
| 10| 1.519538348982754|
| 16|2.3040087637978894|
|  8|1.9324344681638717|
| 10| 2.281251340433997|
|  8|2.3361390224291148|
|  9|2.3125456544357643|
| 10|3.1146497935587423|
| 12| 2.579638377342666|
|  8|2.6836429568994458|
|  7|2.6003487487129417|
|  7|2.8977827343642244|
|  9|3.1896606224913078|
+---+------------------+
only showing top 20 rows



In [94]:
# Модель  `AFTSurvivalRegression`  не подходит для датасета  `CrabAgePrediction`, так как нет информации о том,  
# живы ли были крабы, когда их поймали.

# Эта модель предназначена для анализа данных выживания, где есть данные о времени до события (в данном случае, смерть) 
# и информация о том,  было ли это событие наблюдено (краб умер) или  **цензурировано**  (краб был жив, когда его поймали).  

In [95]:
glr = GeneralizedLinearRegression(featuresCol="features", labelCol="Age", family="gaussian", link="identity", maxIter=10, regParam=0.3)
# Создание конвейера
pipelineGLR = Pipeline(stages = [indexer, assembler, glr])
# Fit the model
modelGLR = pipelineGLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGLR = modelGLR.transform(test).withColumnRenamed("prediction", "predictionsGLR")
Pred_arr.append(predictionsGLR)

In [96]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGLR", metricName = "rmse")
rmseGLR = evaluator.evaluate(predictionsGLR)
tupleRMSE += (rmseGLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseGLR)
# Вывод результата и выключение SparkSession
predictionsGLR.select("Age", "predictionsGLR").show()

Root Mean Squared Error (RMSE): 2.3844774020516617
+---+-----------------+
|Age|   predictionsGLR|
+---+-----------------+
|  9|7.580761401163442|
| 10|7.721660356130881|
|  8|7.594412910373292|
|  9| 7.98521149918198|
|  7|7.967136363977194|
|  7|  7.5298586436638|
| 10| 8.50428846331977|
| 13|8.137992032290327|
| 10|8.165165267636361|
| 16|8.607627993378983|
|  8|  8.2837168226439|
| 10|8.490472642744889|
|  8|8.344835791488114|
|  9| 8.55406860054717|
| 10|8.888309799484986|
| 12|9.163542305264215|
|  8| 8.83083654842655|
|  7|8.694985741934541|
|  7|8.600436230578193|
|  9|8.973276219760443|
+---+-----------------+
only showing top 20 rows



In [97]:
# Создаем списки моделей, выполняющихся том же в порядке выше
modelsList = ["RandomForestRegressor", "GBTRegressor", "LinearRegression", "DecisionTreeRegressor", 
              "IsotonicRegression", "FMRegressor", "GeneralizedLinearRegression"]
shortModelsList = ["predictionsRF", "predictionsGBT", "predictionsLR", "predictionsDT", "predictionsIR", 
                   "predictionsFM", "predictionsGLR"]

# Схема нового DataFrame значений RMSE каждой модели
schemaRMSE = StructType([
    StructField(modelsList[0], DoubleType(), True),
    StructField(modelsList[1], DoubleType(), True),
    StructField(modelsList[2], DoubleType(), True),
    StructField(modelsList[3], DoubleType(), True),
    StructField(modelsList[4], DoubleType(), True),
    StructField(modelsList[5], DoubleType(), True),
    StructField(modelsList[6], DoubleType(), True)
])
# Создание DataFrame
rmseDF = spark.createDataFrame([tupleRMSE], schemaRMSE)
rmseDF.printSchema()

root
 |-- RandomForestRegressor: double (nullable = true)
 |-- GBTRegressor: double (nullable = true)
 |-- LinearRegression: double (nullable = true)
 |-- DecisionTreeRegressor: double (nullable = true)
 |-- IsotonicRegression: double (nullable = true)
 |-- FMRegressor: double (nullable = true)
 |-- GeneralizedLinearRegression: double (nullable = true)



In [98]:
for i, df in enumerate(Pred_arr):
    df = df.withColumn("id", monotonically_increasing_id())
    Pred_arr[i] = df

ListSelectedDF = [df.select("Age", "id", shortModelsList[i]) for i, df in enumerate(Pred_arr)]
predDF = ListSelectedDF[0]

for i in range(1, len(ListSelectedDF)):
    predDF = predDF.join(ListSelectedDF[i], on = ["Age", "id"], how = "inner")

predDF = predDF.drop("id")

predDF.printSchema()
rmseDF.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- predictionsRF: double (nullable = false)
 |-- predictionsGBT: double (nullable = false)
 |-- predictionsLR: double (nullable = false)
 |-- predictionsDT: double (nullable = false)
 |-- predictionsIR: double (nullable = false)
 |-- predictionsFM: double (nullable = false)
 |-- predictionsGLR: double (nullable = false)

root
 |-- RandomForestRegressor: double (nullable = true)
 |-- GBTRegressor: double (nullable = true)
 |-- LinearRegression: double (nullable = true)
 |-- DecisionTreeRegressor: double (nullable = true)
 |-- IsotonicRegression: double (nullable = true)
 |-- FMRegressor: double (nullable = true)
 |-- GeneralizedLinearRegression: double (nullable = true)



In [99]:
predDF.printSchema()
rmseDF.printSchema()
# Вывод лучшего результата
print(f"Лучшая модель: {modelsList[rmseDF.head().index(min(rmseDF.head()))]} (RMSE: {min(rmseDF.head())})")
# Запись результатов в csv-файл
rmseDF.write.csv("RMSE.csv", mode = "overwrite", sep = ";", header = True)
predDF.write.csv("Pred.csv", mode = "overwrite", sep = ";", header = True)

root
 |-- Age: integer (nullable = true)
 |-- predictionsRF: double (nullable = false)
 |-- predictionsGBT: double (nullable = false)
 |-- predictionsLR: double (nullable = false)
 |-- predictionsDT: double (nullable = false)
 |-- predictionsIR: double (nullable = false)
 |-- predictionsFM: double (nullable = false)
 |-- predictionsGLR: double (nullable = false)

root
 |-- RandomForestRegressor: double (nullable = true)
 |-- GBTRegressor: double (nullable = true)
 |-- LinearRegression: double (nullable = true)
 |-- DecisionTreeRegressor: double (nullable = true)
 |-- IsotonicRegression: double (nullable = true)
 |-- FMRegressor: double (nullable = true)
 |-- GeneralizedLinearRegression: double (nullable = true)

Лучшая модель: GBTRegressor (RMSE: 2.3482925159542054)


In [100]:
spark.stop()